In [9]:
print("VS Code + Jupyter is working!")


VS Code + Jupyter is working!


Question 1 - Flexible CNN

This section implements a flexible CNN as required in Question 1.

  5 Conv-Activation-Pool blocks
  Dense + Output layer
  Fully configurable architecture
  Compatible with resized iNaturalist images


In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import wandb

wandb.login()

wandb.init(
    project="DA6401-Assignment2",
    name="cnn-from-scratch",
    config={
        "epochs": 10,
        "batch_size": 64,
        "lr": 0.001
    }
)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FlexibleCNN(nn.Module):
    """
    Assignment Q1: Flexible CNN with 5 convolutional blocks.
    Each block = [Conv2D → Activation → MaxPool2D].
    Followed by 1 dense layer + output layer.
    Supports custom filter sizes, channels, activation function, and dense units.
    Assumes input images are resized to 224x224 (e.g., iNaturalist dataset).
    """
    def __init__(
        self,
        input_channels=3,
        conv_channels=[32, 64, 128, 256, 512],
        kernel_sizes=[3, 3, 3, 3, 3],
        activation_fn=nn.ReLU,
        dense_neurons=256,
        num_classes=101
    ):
        super(FlexibleCNN, self).__init__()

        # Store activation function class and initialize once
        self.activation_fn = activation_fn()

        # Build conv-activation-pool blocks
        layers = []
        in_channels = input_channels

        for out_channels, kernel_size in zip(conv_channels, kernel_sizes):
            layers.append(nn.Conv2d(
                in_channels=in_channels,
                out_channels=out_channels,
                kernel_size=kernel_size,
                padding=kernel_size // 2
            ))
            layers.append(self.activation_fn)
            layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
            in_channels = out_channels

        self.conv_stack = nn.Sequential(*layers)

        # After 5 maxpoolings on 224x224 → size becomes 7x7
        self.flattened_size = conv_channels[-1] * 7 * 7

        self.fc1 = nn.Linear(self.flattened_size, dense_neurons)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(dense_neurons, num_classes)

    def forward(self, x):
        x = self.conv_stack(x)
        x = x.view(x.size(0), -1)                  # Flatten
        x = self.activation_fn(self.fc1(x))        # Dense layer + activation
        x = self.dropout(x)                        # Dropout
        x = self.fc2(x)                            # Output logits
        return x


# import torch
# import torch.nn as nn
# import torch.nn.functional as F

# class SimpleCNN(nn.Module):
#     def __init__(self, num_classes):
#         super(SimpleCNN, self).__init__()

#         # 5 convolutional layers
#         self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)    # [3 → 32]
#         self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)   # [32 → 64]
#         self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)  # [64 → 128]
#         self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1) # [128 → 256]
#         self.conv5 = nn.Conv2d(256, 512, kernel_size=3, padding=1) # [256 → 512]

#         # Max pooling layer (we'll reuse this)
#         self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

#         # Fully connected layers
#         self.fc1 = nn.Linear(512 * 4 * 4, 256)   # 512 channels, 4x4 size after pooling
#         self.dropout = nn.Dropout(0.5)
#         self.fc2 = nn.Linear(256, num_classes)

#     def forward(self, x):
#         x = F.relu(self.conv1(x))               # [32x32]
#         x = self.pool(F.relu(self.conv2(x)))    # [16x16]
#         x = self.pool(F.relu(self.conv3(x)))    # [8x8]
#         x = self.pool(F.relu(self.conv4(x)))    # [4x4]
#         x = self.pool(F.relu(self.conv5(x)))    # [2x2] — optional based on design, we keep at 4x4

#         x = x.view(x.size(0), -1)               # Flatten: [batch, 512*4*4]
#         x = F.relu(self.fc1(x))
#         x = self.dropout(x)
#         x = self.fc2(x)
#         return x


#     def forward(self, x):
#         x = F.relu(self.conv1(x))    # [B, 32, 32, 32]
#         x = self.pool(F.relu(self.conv2(x)))  # [B, 64, 16, 16]
#         x = self.pool(F.relu(self.conv3(x)))  # [B, 128, 8, 8]
        
#         x = x.view(x.size(0), -1)    # Flatten → [B, 128*8*8]
#         x = F.relu(self.fc1(x))
#         x = self.dropout(x)
#         x = self.fc2(x)
#         return x
# model = SimpleCNN(num_classes=10)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)
